In [2]:
import pandas as pd

# Load the datasets
ev_data = pd.read_csv('filtered_charging_stations_data.csv')
charger_data = pd.read_csv('cleaned_charging_station.csv')

# Convert the ID column to strings (if necessary)
charger_data['ID'] = charger_data['ID'].astype(str)

# Function to calculate charging time
def calculate_charging_time(vehicle_model, current_charge, desired_charge, charger_id, temperature=None):
    # Filter the vehicle data based on the model
    vehicle_info = ev_data[ev_data['Model'].str.contains(vehicle_model, case=False, na=False)]

    if vehicle_info.empty:
        return f"Vehicle model '{vehicle_model}' not found in the dataset."

    # Get relevant vehicle details
    battery_capacity_kwh = vehicle_info['Battery: kWh'].values[0]
    ac_charge_rate = vehicle_info['AC Charge Rate (kW)'].values[0]
    dc_charge_rate = vehicle_info['DC Charge Rate (kW)'].values[0]

    # Ensure that charger_id is in the correct format (string)
    charger_id = str(charger_id)

    # Find the charger information based on charger_id
    charger_info = charger_data[charger_data['ID'] == charger_id]

    if charger_info.empty:
        return f"Charger ID '{charger_id}' not found in the dataset."

    charger_power_output = charger_info['Power (kW)'].values[0]
    charger_type = charger_info['Connection Type'].values[0]

    # Use the AC rate or DC rate depending on the charger type
    if charger_type == 'AC':
        effective_charger_power = min(charger_power_output, ac_charge_rate)
    else:
        effective_charger_power = min(charger_power_output, dc_charge_rate)

    # Optional: Adjust efficiency based on environmental conditions
    charging_efficiency = 0.9  # Default value

    if charger_type == 'AC':
        charging_efficiency -= 0.05  # AC chargers are typically less efficient

    if temperature is not None:
        if temperature < 0:
            charging_efficiency -= 0.1  # Decrease efficiency in cold
        elif temperature > 35:
            charging_efficiency -= 0.05  # Decrease efficiency in heat

    # Calculate the percentage of battery needed to be charged
    charge_needed_percentage = desired_charge - current_charge
    if charge_needed_percentage <= 0:
        return f"Current charge is already above or equal to the desired charge level."

    # Calculate the amount of energy (kWh) required to charge the battery to the desired level
    charge_needed_kwh = (charge_needed_percentage / 100) * battery_capacity_kwh

    # Adjust the energy required based on the charging efficiency
    effective_energy_needed_kwh = charge_needed_kwh / charging_efficiency

    # Calculate the time required to charge (in hours)
    charging_time_hours = effective_energy_needed_kwh / effective_charger_power

    # Convert charging time to minutes
    charging_time_minutes = charging_time_hours * 60

    return {
        'Vehicle Model': vehicle_model,
        'Current Charge (%)': current_charge,
        'Desired Charge (%)': desired_charge,
        'Charger Power Output (kW)': charger_power_output,
        'Effective Charger Power (kW)': effective_charger_power,  # Capped by max charge rate
        'Charger Type': charger_type,
        'Charging Efficiency': round(charging_efficiency, 2),
        'Estimated Charging Time (minutes)': round(charging_time_minutes, 2)
    }

# Get user inputs
vehicle_model = input("Enter the vehicle model: ")
current_charge = float(input("Enter the current charge percentage: "))
desired_charge = float(input("Enter the desired charge percentage: "))
charger_id = input("Enter the charger ID: ")
temperature = float(input("Enter the temperature (optional, leave blank for default): ") or "25")  # Default to 25 if left blank

# Calculate and display the charging time
result = calculate_charging_time(vehicle_model, current_charge, desired_charge, charger_id, temperature)
print(result)


Enter the vehicle model: Audi e-tron
Enter the current charge percentage: 20
Enter the desired charge percentage: 80
Enter the charger ID: node/1189771948
Enter the temperature (optional, leave blank for default): 25
{'Vehicle Model': 'Audi e-tron', 'Current Charge (%)': 20.0, 'Desired Charge (%)': 80.0, 'Charger Power Output (kW)': 250.0, 'Effective Charger Power (kW)': 150, 'Charger Type': 'DC', 'Charging Efficiency': 0.9, 'Estimated Charging Time (minutes)': 25.33}
